In [8]:
import sqlite3
import pandas as pd 

In [4]:
# Kết nối SQLite (tạo file database trong bộ nhớ hoặc lưu vào file)
conn = sqlite3.connect("students.db")  # Tạo file database students.db
cursor = conn.cursor()

In [5]:
cursor.execute("DROP TABLE IF EXISTS student;")
cursor.execute("DROP TABLE IF EXISTS course;")

#Tạo bảng student
cursor.execute("""
CREATE TABLE IF NOT EXISTS student (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    class TEXT,
    course_id INTEGER,
    score REAL
);
""")

# Tạo bảng course
cursor.execute("""
CREATE TABLE IF NOT EXISTS course (
    id INTEGER PRIMARY KEY,
    course_name TEXT
);
""")

# Lưu thay đổi vào database
conn.commit()


In [6]:
# Dữ liệu cho bảng student (giữ nguyên None nếu thiếu course_id)
student_data = [
    (1, "Nguyen Minh Hoang", "May Tinh", 12, 6.7),
    (2, "Tran Thi Lan", "Kinh Te", 34, 9.2),
    (3, "Pham Van Nam", "Toan Tin", None, 7.9),  # Thiếu course_id
    (4, "Le Thanh Huyen", "Toan Tin", 20, 7.2),
    (5, "Vu Quoc Anh", "May Tinh", 24, 8.0),
    (6, "Dang Thuy Linh", "May Tinh", 24, 5.5),
    (7, "Bui Tien Dung", "Kinh Te", 34, 9.2),
    (8, "Ho Ngoc Mai", "Toan Tin", 20, 8.8),
    (9, "Duong Huu Phuc", "Kinh Te", None, 7.2),  # Thiếu course_id
    (10, "Cao Thi Hanh", "May Tinh", None, 7.0)   # Thiếu course_id
]

# Dữ liệu cho bảng course
course_data = [
    (12, "Giai tich"),
    (34, "Thong ke"),
    (26, "Tin hoc")
]
# Chèn dữ liệu vào bảng student và course
cursor.executemany("INSERT OR IGNORE INTO student VALUES (?, ?, ?, ?, ?);", student_data)
cursor.executemany("INSERT OR IGNORE INTO course VALUES (?, ?);", course_data)
conn.commit()

In [7]:
# Đọc dữ liệu từ bảng student để kiểm tra
df_student = pd.read_sql_query("SELECT * FROM student;", conn)
df_course = pd.read_sql_query("SELECT * FROM course;", conn)

# Hiển thị kết quả
print("Bảng Student:")
display(df_student)

print("Bảng Course:")
display(df_course)

# Đóng kết nối
conn.close()

Bảng Student:


,student_id,name,class,course_id,score
0,1,Nguyen Minh Hoang,May Tinh,12.0,6.7
1,2,Tran Thi Lan,Kinh Te,34.0,9.2
2,3,Pham Van Nam,Toan Tin,NaN,7.9
3,4,Le Thanh Huyen,Toan Tin,20.0,7.2
4,5,Vu Quoc Anh,May Tinh,24.0,8.0
5,6,Dang Thuy Linh,May Tinh,24.0,5.5
6,7,Bui Tien Dung,Kinh Te,34.0,9.2
7,8,Ho Ngoc Mai,Toan Tin,20.0,8.8
8,9,Duong Huu Phuc,Kinh Te,NaN,7.2
9,10,Cao Thi Hanh,May Tinh,NaN,7.0


Bảng Course:


,id,course_name
0,12,Giai tich
1,26,Tin hoc
2,34,Thong ke


# JOIN bảng Student và Course:

In [11]:
df_inner = df_student.merge(df_course, left_on='course_id', right_on='id', how='inner')
print("INNER JOIN:\n", df_inner)


INNER JOIN:
    student_id               name     class  course_id  score  id course_name
0           1  Nguyen Minh Hoang  May Tinh       12.0    6.7  12   Giai tich
1           2       Tran Thi Lan   Kinh Te       34.0    9.2  34    Thong ke
2           7      Bui Tien Dung   Kinh Te       34.0    9.2  34    Thong ke


In [13]:
df_right = df_student.merge(df_course, left_on='course_id', right_on='id', how='right')
print("RIGHT JOIN:\n", df_right)


RIGHT JOIN:
    student_id               name     class  course_id  score  id course_name
0         1.0  Nguyen Minh Hoang  May Tinh       12.0    6.7  12   Giai tich
1         NaN                NaN       NaN        NaN    NaN  26     Tin hoc
2         2.0       Tran Thi Lan   Kinh Te       34.0    9.2  34    Thong ke
3         7.0      Bui Tien Dung   Kinh Te       34.0    9.2  34    Thong ke


# 2 Hãy cập nhật những giá trị course_id còn thiếu trong bảng studenttrong đó các giá trị được điền là những giá trị nằm trong bảng course và loại bỏ nhữngbản ghi tham gia những môn học không tồn tại bảng course.

In [17]:
import numpy as np

# Điền các giá trị course_id bị thiếu bằng giá trị hợp lệ bất kỳ từ df_course
valid_course_ids = df_course['id'].tolist()
df_student['course_id'] = df_student['course_id'].apply(lambda x: np.random.choice(valid_course_ids) if pd.isna(x) else x)

# Loại bỏ sinh viên có course_id không tồn tại trong bảng Course
df_student = df_student[df_student['course_id'].isin(df_course['id'])]

print("Bảng Student sau khi cập nhật course_id:\n", df_student)


Bảng Student sau khi cập nhật course_id:
    student_id               name     class  course_id  score
0           1  Nguyen Minh Hoang  May Tinh       12.0    6.7
1           2       Tran Thi Lan   Kinh Te       34.0    9.2
2           3       Pham Van Nam  Toan Tin       26.0    7.9
6           7      Bui Tien Dung   Kinh Te       34.0    9.2
8           9     Duong Huu Phuc   Kinh Te       26.0    7.2
9          10       Cao Thi Hanh  May Tinh       12.0    7.0


+ a. Tổng số sinh viên, điểm trung bình của từng lớp.

In [18]:
df_class_stats = df_student.groupby('class').agg(
    total_students=('student_id', 'count'),
    avg_score=('score', 'mean')
).reset_index()

print("Tổng số sinh viên và điểm trung bình theo từng lớp:\n", df_class_stats)


Tổng số sinh viên và điểm trung bình theo từng lớp:
       class  total_students  avg_score
0   Kinh Te               3   8.533333
1  May Tinh               2   6.850000
2  Toan Tin               1   7.900000


+ b,Tổng số sinh viên, điểm trung bình của từng môn học

In [20]:
df_merged = df_student.merge(df_course, left_on='course_id', right_on='id')

df_course_stats = df_merged.groupby('course_name').agg(
    total_students=('student_id', 'count'),
    avg_score=('score', 'mean')
).reset_index()

print("Tổng số sinh viên và điểm trung bình của  từng môn học:\n", df_course_stats)


Tổng số sinh viên và điểm trung bình của  từng môn học:
   course_name  total_students  avg_score
0   Giai tich               2       6.85
1    Thong ke               2       9.20
2     Tin hoc               2       7.55


+ c, Phân loại thi đua theo số điểm của từng môn học biết:

In [21]:
def classify(score):
    if score >= 9.0:
        return 'Xuất sắc'
    elif 6.0 <= score <= 8.9:
        return 'Tốt'
    else:
        return 'Kém'

df_course_stats['classification'] = df_course_stats['avg_score'].apply(classify)

print("Phân loại thi đua theo từng môn học:\n", df_course_stats)


Phân loại thi đua theo từng môn học:
   course_name  total_students  avg_score classification
0   Giai tich               2       6.85            Tốt
1    Thong ke               2       9.20       Xuất sắc
2     Tin hoc               2       7.55            Tốt


# 3. Hãy xếp hạng sinh viên thông qua:

+ a. Điểm số:

In [23]:
df_student['rank_overall'] = df_student['score'].rank(method='dense', ascending=False)
df_student = df_student.sort_values(by='rank_overall')
print("Xếp hạng sinh viên theo điểm số chung:\n", df_student[['student_id', 'name', 'score', 'rank_overall']])


Xếp hạng sinh viên theo điểm số chung:
    student_id               name  score  rank_overall
1           2       Tran Thi Lan    9.2           1.0
6           7      Bui Tien Dung    9.2           1.0
2           3       Pham Van Nam    7.9           2.0
8           9     Duong Huu Phuc    7.2           3.0
9          10       Cao Thi Hanh    7.0           4.0
0           1  Nguyen Minh Hoang    6.7           5.0


+ b. Điểm số theo lớp học:


In [24]:
df_student['rank_by_class'] = df_student.groupby('class')['score'].rank(method='dense', ascending=False)
df_student = df_student.sort_values(by=['class', 'rank_by_class'])
print("Xếp hạng sinh viên theo điểm số trong từng lớp:\n", df_student[['student_id', 'name', 'class', 'score', 'rank_by_class']])


Xếp hạng sinh viên theo điểm số trong từng lớp:
    student_id               name     class  score  rank_by_class
1           2       Tran Thi Lan   Kinh Te    9.2            1.0
6           7      Bui Tien Dung   Kinh Te    9.2            1.0
8           9     Duong Huu Phuc   Kinh Te    7.2            2.0
9          10       Cao Thi Hanh  May Tinh    7.0            1.0
0           1  Nguyen Minh Hoang  May Tinh    6.7            2.0
2           3       Pham Van Nam  Toan Tin    7.9            1.0


+ c,Điểm số theo mã môn học :

In [25]:
df_student['rank_by_course'] = df_student.groupby('course_id')['score'].rank(method='dense', ascending=False)
df_student = df_student.sort_values(by=['course_id', 'rank_by_course'])
print("Xếp hạng sinh viên theo điểm số trong từng môn học:\n", df_student[['student_id', 'name', 'course_id', 'score', 'rank_by_course']])


Xếp hạng sinh viên theo điểm số trong từng môn học:
    student_id               name  course_id  score  rank_by_course
9          10       Cao Thi Hanh       12.0    7.0             1.0
0           1  Nguyen Minh Hoang       12.0    6.7             2.0
2           3       Pham Van Nam       26.0    7.9             1.0
8           9     Duong Huu Phuc       26.0    7.2             2.0
1           2       Tran Thi Lan       34.0    9.2             1.0
6           7      Bui Tien Dung       34.0    9.2             1.0


In [27]:
top_3_highest_overall = df_student.nsmallest(3, 'rank_overall')
top_3_lowest_overall = df_student.nlargest(3, 'rank_overall')

print(" Top 3 sinh viên có điểm cao nhất toàn bộ:\n", top_3_highest_overall[['student_id', 'name', 'score', 'rank_overall']])
print("\n Top 3 sinh viên có điểm thấp nhất toàn bộ:\n", top_3_lowest_overall[['student_id', 'name', 'score', 'rank_overall']])


 Top 3 sinh viên có điểm cao nhất toàn bộ:
    student_id           name  score  rank_overall
1           2   Tran Thi Lan    9.2           1.0
6           7  Bui Tien Dung    9.2           1.0
2           3   Pham Van Nam    7.9           2.0

 Top 3 sinh viên có điểm thấp nhất toàn bộ:
    student_id               name  score  rank_overall
0           1  Nguyen Minh Hoang    6.7           5.0
9          10       Cao Thi Hanh    7.0           4.0
8           9     Duong Huu Phuc    7.2           3.0


# 4.  BBổ sung thêm một trường graduation_date có kiểu dữ liệu là DATETIME vào bảngstudent để xác định thời gian tốt nghiệp của từng bạn, trong đó thời gian tốt nghiệp đượcxác định bởi thời gian hiện tại cộng với số hạng tương ứng của bạn đó tính theo điểm số:

In [28]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Lấy ngày hiện tại
current_date = datetime.now()

# Thêm cột graduation_date: ngày hiện tại + (số thứ hạng * 30 ngày)
df_student['graduation_date'] = current_date + df_student['rank_overall'].apply(lambda x: timedelta(days=x * 30))

# Hiển thị kết quả
print(df_student[['student_id', 'name', 'score', 'rank_overall', 'graduation_date']])


   student_id               name  score  rank_overall  \
9          10       Cao Thi Hanh    7.0           4.0   
0           1  Nguyen Minh Hoang    6.7           5.0   
2           3       Pham Van Nam    7.9           2.0   
8           9     Duong Huu Phuc    7.2           3.0   
1           2       Tran Thi Lan    9.2           1.0   
6           7      Bui Tien Dung    9.2           1.0   

             graduation_date  
9 2025-07-08 14:18:21.061879  
0 2025-08-07 14:18:21.061879  
2 2025-05-09 14:18:21.061879  
8 2025-06-08 14:18:21.061879  
1 2025-04-09 14:18:21.061879  
6 2025-04-09 14:18:21.061879  


Sinh viên có rank 1 (cao nhất, điểm 9.2) tốt nghiệp sớm nhất vào 09/04/2025.

Sinh viên có rank 2 tốt nghiệp sau đó một tháng vào 09/05/2025.

Sinh viên có rank 3, 4, 5 tiếp tục tốt nghiệp muộn hơn, mỗi người cách nhau khoảng 30 ngày.

In [29]:
df_student[['student_id', 'name', 'score', 'rank_overall', 'graduation_date']].sort_values(by='rank_overall')


,student_id,name,score,rank_overall,graduation_date
6,7,Bui Tien Dung,9.2,1.0,2025-04-09 14:18:21.061879
1,2,Tran Thi Lan,9.2,1.0,2025-04-09 14:18:21.061879
2,3,Pham Van Nam,7.9,2.0,2025-05-09 14:18:21.061879
8,9,Duong Huu Phuc,7.2,3.0,2025-06-08 14:18:21.061879
9,10,Cao Thi Hanh,7.0,4.0,2025-07-08 14:18:21.061879
0,1,Nguyen Minh Hoang,6.7,5.0,2025-08-07 14:18:21.061879
